<a href="https://colab.research.google.com/github/srilalithaveerubhotla/Trash-Management-System/blob/master/trashmanagement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import cv2
import numpy
import glob
import pylab as plt
import os

# importing libraries
from matplotlib import pyplot
from keras import datasets
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.utils import to_categorical
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


from keras.callbacks import TensorBoard

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
from packaging import version


import numpy as np
import cv2
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.layers import Conv2D, Flatten, MaxPooling2D,Dense,Dropout,SpatialDropout2D
from keras.models  import Sequential
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
import random,os,glob
import matplotlib.pyplot as plt

#from kerastuner.tuners import RandomSearch
from keras.layers import Input, Dense
from keras.models import Model
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from tensorboard.plugins.hparams import api as hp
from keras.utils import print_summary, to_categorical

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."
from datetime import datetime
%load_ext tensorboard
# %reload_ext tensorboard
%tensorboard --logdir {logs_base_dir}

In [0]:
from google.colab import drive
drive.mount('/content/drive')


In [0]:
!unzip /content/drive/My\ Drive/garbage-classification.zip

In [0]:
# Importing and deriving the training images and labelling them
trash_images = []
labels = [] 
for fruit_dir_path in glob.glob("/content/garbage classification/Garbage classification/*/"):
    fruit_label = fruit_dir_path.split("/")[4]
    for image_path in glob.glob(os.path.join(fruit_dir_path, "*.jpg")):
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        
        image = cv2.resize(image, (70, 70))
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        trash_images.append(image)
        labels.append(fruit_label)
trash_images = np.array(trash_images)
labels = np.array(labels)

In [0]:
labels

In [0]:
label_to_id_dict = {v:i for i,v in enumerate(np.unique(labels))}
id_to_label_dict = {v: k for k, v in label_to_id_dict.items()}

In [0]:
training_label_id = np.array([label_to_id_dict[i] for i in labels])

In [0]:
id_to_label_dict

In [0]:
print("Train_data shape......:",trash_images.shape)
print("Labels shape..........:",training_label_id.shape)

In [0]:
(trainX, testX, trainY, testY) = train_test_split(trash_images, training_label_id, test_size=0.25)

In [0]:
now = datetime.now()
logdir = "logs/trash_images/" + now.strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir + "/metrics")
with file_writer.as_default():
  tf.summary.image("Training data", np.reshape(trainX[0:10],(-2,70,70,3)), step=0)
file_writer.set_as_default()

In [0]:
%tensorboard --logdir logs/trash_images/

In [0]:
dir_path = '/content/garbage classification/Garbage classification'

In [0]:
img_list = glob.glob(os.path.join(dir_path, '*/*.jpg'))

In [0]:
len(img_list)


In [0]:
# img_list

In [0]:
# img_list

In [0]:
# for i, img_path in enumerate(random.sample(img_list, 6)):
#     img = image.load_img(img_path, target_size=(224, 224))
#     img = image.img_to_array(img, dtype=np.uint8)
#     plt.subplot(2, 3, i+1)
#     plt.imshow(img.squeeze())

In [0]:

train=ImageDataGenerator(horizontal_flip=True,
                         vertical_flip=True,
                         validation_split=0.1,
                         rescale=1./255,
                         shear_range = 0.1,
                         zoom_range = 0.1,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,)

test=ImageDataGenerator(rescale=1/255,
                        validation_split=0.1)

train_generator=train.flow_from_directory(dir_path,
                                          target_size=(300,300),
                                          batch_size=32,
                                          class_mode='categorical',
                                          subset='training')

test_generator=test.flow_from_directory(dir_path,
                                        target_size=(300,300),
                                        batch_size=32,
                                        class_mode='categorical',
                                        subset='validation')

validation_generator = test.flow_from_directory(
    dir_path,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='validation',
    seed=0
)

labels = (train_generator.class_indices)
print(labels)

labels = dict((v,k) for k,v in labels.items())
print(labels)


In [0]:
for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape

In [0]:
print (train_generator.class_indices)

Labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(Labels)

## Simple CNN Model 

In [0]:
import tensorflow as tf

tf.random.set_seed(42)
np.random.seed(42)

model=tf.keras.Sequential()
#Convolution blocks

model.add(tf.keras.layers.Conv2D(32,(3,3), padding='same',input_shape=(300,300,3),activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2)) 

model.add(tf.keras.layers.Conv2D(64,(3,3), padding='same',activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2)) 

model.add(tf.keras.layers.Conv2D(32,(3,3), padding='same',activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2)) 

#Classification layers
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(64,activation='relu'))
#model.add(SpatialDropout2D(0.5))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(32,activation='relu'))

model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(6,activation='softmax'))

filepath="/content/trained_model.h5"
logdir="logs/trash_images/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint("trash_simple_model.h5", save_best_only=True)
callbacks_list = [model_checkpoint,tensorboard_callback]

In [0]:
model.summary()

In [0]:
keras.utils.plot_model(model, "simple_cnn_trash.png", show_shapes=True)

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc','mse']) # RMS PROP - No accuracy

#es=EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)


In [0]:

history = model.fit_generator(train_generator,
                              epochs=10,
                              steps_per_epoch=2276//32,
                              validation_data=test_generator,
                              validation_steps=251//32,
                              workers = 4,
                              callbacks=callbacks_list) 

In [0]:
!tensorboard dev upload --logdir logs/trash_images/ \
  --name "Trash Management" \
  --description "Training results from https://colab.sandbox.google.com/github/tensorflow/tensorboard/blob/master/docs/tbdev_getting_started.ipynb"

## Simple CNN Fine Tuned Model

In [0]:
model=tf.keras.Sequential()
#Convolution blocks

model.add(tf.keras.layers.Conv2D(32,(3,3), padding='same',input_shape=(300,300,3),activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2)) 
model.add(tf.keras.layers.Dropout(0.5)) # No accuracy

model.add(tf.keras.layers.Conv2D(64,(3,3), padding='same',activation='selu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2)) 
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Conv2D(32,(3,3), padding='same',activation='selu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2)) 

#Classification layers
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(126,activation='selu'))
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(64,activation='selu'))

model.add(tf.keras.layers.Dropout(0.5))


model.add(tf.keras.layers.Dense(32,activation='selu'))
#Classification layers
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(16,activation='selu'))
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(6,activation='softmax'))

filepath="/content/trained_model.h5"
logdir="logs/trash_images/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint("trash_simple_model1.h5", save_best_only=True)
callbacks_list = [model_checkpoint,tensorboard_callback]

In [0]:
model.summary()

In [0]:
keras.utils.plot_model(model, "simple_cnn1_trash.png", show_shapes=True)

In [0]:
model.compile(loss='kullback_leibler_divergence', optimizer='nadam', metrics=['acc'])


In [0]:
history = model.fit_generator(train_generator,
                              epochs=10,
                              steps_per_epoch=2276//32,
                              validation_data=test_generator,
                              validation_steps=251//32,
                              workers = 4,
                              callbacks=callbacks_list) 

In [0]:
!mkdir -p classifier1

In [0]:
model_save_name = 'classifier1.h5'
model.save('/content/trained_model.h5')

In [0]:
model_json = model.to_json()
with open("/content/model.json", "w") as json_file:
    json_file.write(model_json)

### VGG16 TRANSFER LEARNING Using Pretrained

In [0]:
from keras.applications.vgg16 import VGG16
from keras.utils.vis_utils import plot_model

In [0]:
model = VGG16()
plot_model(model, to_file='vgg.png')

In [0]:
model = VGG16()
print(model.summary())

In [0]:
from keras.preprocessing.image import load_img
# load an image from file
image = load_img('/content/garbage classification/Garbage classification/paper/paper357.jpg', target_size=(224, 224))


In [0]:
from keras.preprocessing.image import img_to_array
# convert the image pixels to a numpy array
image = img_to_array(image)

In [0]:
# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

In [0]:
from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image = preprocess_input(image)

In [0]:
yhat = model.predict(image)

In [0]:
from keras.applications.vgg16 import decode_predictions
# convert the probabilities to class labels
label = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
label = label[0][0]
# print the classification
print('%s (%.2f%%)' % (label[1], label[2]*100))

In [0]:
!unzip /content/drive/My\ Drive/keras-pretrained-models.zip

In [0]:
gen = {
    "train": ImageDataGenerator(
        horizontal_flip=True,
        vertical_flip=True,
        rescale=1. / 255,
        validation_split=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        width_shift_range=0.1,
        height_shift_range=0.1,
        rotation_range=30,
    ).flow_from_directory(
        directory=dir_path,
        target_size=(300, 300),
        subset='training',
    ),

    "valid": ImageDataGenerator(
        rescale=1 / 255,
        validation_split=0.1,
    ).flow_from_directory(
        directory=dir_path,
        target_size=(300, 300),
        subset='validation',
    ),
}

## VGG 16 From Scratch code

## INCEPTION V3 TOP Notch Transfer Learned

In [0]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import InceptionV3
from keras_preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras import optimizers, losses
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint
import pickle


In [0]:
base_model = InceptionV3(weights=None, include_top=False, input_shape=(300, 300, 3))
base_model.load_weights('/content/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
base_model.trainable = False

In [0]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.15),
    Dense(1024, activation='relu'),
    Dense(6, activation='softmax')
])

opt = optimizers.nadam(lr=0.0001)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy'])

batch_size = 200
epochs = 10
train_generator = gen["train"]
valid_generator = gen["valid"]

steps_per_epoch = train_generator.n // batch_size
validation_steps = valid_generator.n // batch_size

filepath = "model_{epoch:02d}-{val_accuracy:.2f}.h5"
checkpoint1 = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint1]
history = model.fit_generator(generator=train_generator, epochs=epochs, steps_per_epoch=steps_per_epoch,
                              validation_data=valid_generator, validation_steps=validation_steps,
                              callbacks=callbacks_list)
with open('trainHistoryDict.txt', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

## RESNET 50 top notch Transfer Learning

In [0]:
from keras.applications.vgg16 import VGG16
from keras_preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras import optimizers, losses
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint
import pickle


In [0]:
base_model = VGG16(weights=None, include_top=False, input_shape=(300, 300, 3))
base_model.load_weights('/content/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')
base_model.trainable = False

In [0]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.15),
    Dense(1024, activation='relu'),
    Dense(6, activation='softmax')
])

opt = optimizers.nadam(lr=0.0001)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy'])

batch_size = 200
epochs = 10
train_generator = gen["train"]
valid_generator = gen["valid"]

steps_per_epoch = train_generator.n // batch_size
validation_steps = valid_generator.n // batch_size

filepath = "model_{epoch:02d}-{val_accuracy:.2f}.h5"
checkpoint1 = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint1]
history = model.fit_generator(generator=train_generator, epochs=epochs, steps_per_epoch=steps_per_epoch,
                              validation_data=valid_generator, validation_steps=validation_steps,
                              callbacks=callbacks_list)
with open('trainHistoryDict.txt', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

## RESNET 50 SCRATCH CODE

In [0]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [0]:
def identity_block(X, f, filters, stage, block):
    
    # Defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1, 1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path 
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1, 1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [0]:
def convolutional_block(X, f, filters, stage, block, s=2):

    # Defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X

    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### 
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [0]:
def ResNet50(input_shape = (64, 64, 3), classes = 6):
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL
    X = AveragePooling2D(pool_size=(2,2), padding='same')(X)

    # Output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [0]:
# Normalize image vectors
X_train = trainX/255.
X_test = testX/255.

# Convert training and test labels to one hot matrices
Y_train = to_categorical(trainY, 6)
Y_test = to_categorical(testY, 6)

In [0]:
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

In [0]:
model = ResNet50(input_shape = (70, 70, 3), classes = 6)
keras.utils.plot_model(model, "simple_RESNET_trash.png", show_shapes=True)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [0]:
model.fit(X_train,Y_train, epochs = 2)

## MobileNet Transfer Learned Top Notch

In [0]:
IMG_SHAPE = (224,224,3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False, 
                                               weights='imagenet')
base_model.trainable = False


In [0]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(128, 3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(6, activation='softmax')
])

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), #Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


In [0]:
batch_size = 32
epochs = 50
steps_per_epoch = train_generator.n // batch_size
validation_steps = validation_generator.n // batch_size

history = model.fit_generator(train_generator, 
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs, 
                              workers=4,
                              validation_data=validation_generator, 
                              validation_steps=validation_steps)


## MOBILE NET From Scratch code 

In [0]:
# train_X = np.array( trainX ) / 255
# train_Y = np.array( trainY )
# test_X = np.array( testX ) / 255
# test_Y = np.array( testY )

# train_Y = tf.keras.utils.to_categorical( train_Y , num_classes=6 )
# test_Y = tf.keras.utils.to_categorical( test_Y , num_classes=6 )

In [0]:
# train_X.shape

In [0]:

# Note: You may use tf.keras.layers.DepthwiseConv2D but you won't be able to add BatchNorm and LeakyReLU layers.
# Hence, we are first performing depthwise convolutions and then a Conv2D with kernel size of 1.
def SeparableConv( x , num_filters , strides , alpha=1.0 ):
    x = tf.keras.layers.DepthwiseConv2D( kernel_size=3 , padding='same' )( x )
    x = tf.keras.layers.BatchNormalization(momentum=0.9997)( x )
    x = tf.keras.layers.Activation( 'relu' )( x )
    x = tf.keras.layers.Conv2D( np.floor( num_filters * alpha ) , kernel_size=( 1 , 1 ) , strides=strides , use_bias=False , padding='same' )( x )
    x = tf.keras.layers.BatchNormalization(momentum=0.9997)(x)
    x = tf.keras.layers.Activation('relu')(x)
    return x

def Conv( x , num_filters , kernel_size , strides=1 , alpha=1.0 ):
    x = tf.keras.layers.Conv2D( np.floor( num_filters * alpha ) , kernel_size=kernel_size , strides=strides , use_bias=False , padding='same' )( x )
    x = tf.keras.layers.BatchNormalization( momentum=0.9997 )(x)
    x = tf.keras.layers.Activation('relu')(x)
    return x

# The number of classes are three.
num_classes = 6

# The shape of the input image.
inputs = tf.keras.layers.Input( shape=( 70 , 70 , 3))

x = Conv( inputs , num_filters=32 , kernel_size=3 , strides=2 )
x = SeparableConv( x , num_filters=32 , strides=1 )
x = Conv( x , num_filters=64 , kernel_size=1 )
x = SeparableConv( x , num_filters=64 , strides=2  )
x = Conv( x , num_filters=128 , kernel_size=1 )
x = SeparableConv( x , num_filters=128 , strides=1  )
x = Conv( x , num_filters=128 , kernel_size=1 )
x = SeparableConv( x , num_filters=128 , strides=2  )
x = Conv( x , num_filters=256 , kernel_size=1 )
x = SeparableConv( x , num_filters=256 , strides=1  )
x = Conv( x , num_filters=256 , kernel_size=1 )
x = SeparableConv( x , num_filters=256 , strides=2  )
x = Conv( x , num_filters=512 , kernel_size=1 )

# You may uncomment the code below if you're machine could tolerate such heavy computation!
#for i in range( 5 ):
    #x = SeparableConv(x, num_filters=512 , strides=1 )
    #x = Conv(x, num_filters=512 , kernel_size=1 )

x = SeparableConv(x, num_filters=512 , strides=2 )
x = Conv(x, num_filters=1024 , kernel_size=1 )
x = tf.keras.layers.AveragePooling2D( pool_size=( 7 , 7 ))( x )
x = tf.keras.layers.Flatten()( x )
x = tf.keras.layers.Dense( num_classes )( x )
outputs = tf.keras.layers.Activation( 'softmax' )( x )

model = tf.keras.models.Model( inputs , outputs )

# As we doing classification, we'll use categorical crossentropy and the RMSProp optimizier.
model.compile( loss='categorical_crossentropy' , optimizer='rmsprop' , metrics=[ 'acc' ] )


In [0]:
keras.utils.plot_model(model, "simple_cnn1_trash.png", show_shapes=True)

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [0]:

model.fit(trainX, trainY, epochs=5)


In [0]:
# # !git clone https://github.com/pjreddie/darknet
# !cd darknet/
# !make
# !wget https://pjreddie.com/media/files/yolov3.weights